In [1]:
import pandas as pd
import numpy as np
import cobra
from CBModellingFuncs import *
import time
import os
import threadpoolctl

#### Important - define number of processes
processes = 50
####


threadpoolctl.threadpool_limits(limits=processes)
os.makedirs("../results/sampling/thinning_test", exist_ok=True)
os.makedirs("../results/dataframes/sampling", exist_ok=True)
cobra_config = cobra.Configuration()
cobra_config.solver = "cplex"

In [2]:
model, m_warnings = cobra.io.validate_sbml_model("../results/iMT1026-NZ.xml")

model.solver = 'cplex'
model.tolerance = 1e-6


rgdict = {}
for g in model.groups:
    for r in g.members:
        
        rgdict[r.id] = g.id.title()

SBML errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


In [3]:
DRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/DerivedRetentInfo.csv", index_col=0, header=[0,1])
IRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/InterpolatedRetentInfo.csv", index_col=0, header=[0,1])
FRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/FittedRetentInfo.csv", index_col=0, header=[0,1])

NGAMseries = pd.read_csv('../results/dataframes/NGAMseries.csv',index_col=0,squeeze=True)

model_viab_cols = ["Growth Rate model 1/h", "qS model mmol/gh","qP model g/gh", "qCO2 viable model mmol/gh", "qO2 viable model mmol/gh", "qStorGlyc mmol/gh"]
viab_cols = ["Average Growth Rate 1/h", "qS viable mmol/gh","qP g/gh", "qCO2 viable mmol/gh", "qO2 viable mmol/gh", "qStorGlyc mmol/gh"] 

## Sampling

In [4]:
co2=True
o2=True

- Uncomment the below cell to execute and produce the sampled chains

In [5]:
%%time
run_time = []
sampling_results = []

equation_names = ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]
datasets = [DRinfo, DRinfo, IRinfo, FRinfo, DRinfo]

for tfactor in [1,10,100, 1000]:
    for i, equation in enumerate(equation_names):
        data = datasets[i].loc[:, model_viab_cols].copy()
        if equation in ["Consensus", "ScaledConsensus"]:
            data["qStorGlyc mmol/gh"] = 0
        
        time_0 = data.iloc[0,:]
        name = time_0.name
        values = time_0
        mu = values[0]
        sd = values[1]
    
        loop_start = time.time()


        with model as temp_model:
            
            set_biomass_objective(temp_model, equation, name)

            temp_model.reactions.ATPM.bounds = (NGAMseries[f"{equation}"],1000)
           
            solution = constrain_predict(temp_model, values, co2=co2)
           
            pred_mu = solution.objective_value
            
            if equation in ["Consensus", "ScaledConsensus"]:
                temp_model.reactions.get_by_id(equation).bounds = (pred_mu*0.95, pred_mu)
                
            else:
                temp_model.reactions.get_by_id(f"{equation}Biomass{name}").bounds = (pred_mu*0.95, pred_mu)

            flux_span = constrain_predict(temp_model, values, co2=co2, fluxva=True, processes=processes,fopt=0)
            remove_blocked(temp_model, flux_span, solution)


            chains = constrain_predict(temp_model, values, sampling=True, chains=4, thinning=tfactor, 
                                       loopless=True, processes=processes, co2=co2, n_samples=1250)

                 # ensure all have same number of samples, may have
            chains = [df.iloc[:min([df.shape[0] for df in chains]), :] for df in chains]
            print(f"{equation} chains with thinning factor of  {tfactor} all have length of {chains[0].shape[0]}") 

            for j, chain in enumerate(chains):
                chain["Chain"] = j+1
            chains = pd.concat(chains, ignore_index=True)
            chains.to_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz", 
                                compression={'method': 'gzip', 'compresslevel': 6, 'mtime': 1})
            
        loop_end = time.time()
        run_time.append((equation, tfactor, (loop_end - loop_start)/60))
                         
run_time = pd.DataFrame(run_time, columns=["Equation", "Thinning", "Time"])
run_time.to_csv("../results/dataframes/sampling/Thinning_RunTime.csv")

2387 reactions before making consistent, 1425 after
Consensus chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1437 after
Derived chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1435 after
Interpolated chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1424 after
Fitted chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1442 after
ScaledConsensus chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1428 after
Consensus chains with thinning factor of  10 all have length of 1250
2387 reactions before making consistent, 1439 after
Derived chains with thinning factor of  10 all have length of 1250
2387 reactions before making consistent, 1440 after
Interpolated chains with thinning factor of  10 all have length of 1250
2387 reactions before making con

In [6]:
%%time

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

all_stats = pd.DataFrame()
n_chains = 4

for tfactor in [1,10,100, 1000]:#, 10000]:
    
    con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestConsensus{name}T{tfactor}.pkl.gz")
    con_chains = [con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    scaled_con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestScaledConsensus{name}T{tfactor}.pkl.gz")
    scaled_con_chains = [scaled_con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    der_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestDerived{name}T{tfactor}.pkl.gz")
    der_chains = [der_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
   
    int_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestInterpolated{name}T{tfactor}.pkl.gz")
    int_chains = [int_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    fit_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestFitted{name}T{tfactor}.pkl.gz")
    fit_chains = [fit_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    shared_columns = list(set(con_chains[0].columns).intersection(scaled_con_chains[0].columns, 
                                                                  der_chains[0].columns, 
                                                                  int_chains[0].columns, 
                                                                  fit_chains[0].columns))
    
    con_chains = [df.loc[:,shared_columns] for df in con_chains]
    scaled_con_chains = [df.loc[:,shared_columns] for df in scaled_con_chains]
    der_chains = [df.loc[:,shared_columns] for df in der_chains]
    int_chains = [df.loc[:,shared_columns] for df in int_chains]
    fit_chains = [df.loc[:,shared_columns] for df in fit_chains]
 
    all_df = []

    for tup in [("Consensus", con_chains),
                ("ScaledConsensus", scaled_con_chains),
                ("Derived", der_chains),
                ("Interpolated", int_chains),
                ("Fitted", fit_chains)]:
    
        biocomp, model_chains = tup
        grouped, rxns = extractchains(model_chains)
        
        for_pool = list(zip(grouped,rxns))
      
        pool = Pool(processes=processes)
        all_series = pool.starmap(calculateDiagnostics, for_pool)
        pool.close()
        df = pd.concat(all_series,axis=1).T
       
        df["Biomass Composition"] = biocomp
        all_df.append(df)
        
    all_df = pd.concat(all_df, ignore_index=True)
    all_df["Thinning Factor"] = tfactor
    
    all_stats = pd.concat([all_stats, all_df], ignore_index=True)
        
all_stats.set_index(["Thinning Factor", "Biomass Composition"], inplace=True)
all_stats["Rhat Fails"] = all_stats["Rhat"] > 1.01
all_stats["ESS-Bulk Fails"] = all_stats["ESS-Bulk"] < 400
n_grouped_chains = all_stats.groupby(["Thinning Factor", "Biomass Composition"]).count()["Rxn"]


# summary_stats = all_stats[["Geweke Fails","ESS Split Fails", "Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = all_stats[["Geweke Fails","Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = (summary_stats > 0).groupby(["Thinning Factor", "Biomass Composition"]).sum()
temp_df = (summary_stats.divide(n_grouped_chains, axis=0) * 100)
# temp_df.columns=["% Geweke Fails","% seperated-ESS Fails","% Rhat Fails","% bulk-ESS Fails"]
temp_df.columns=["% Geweke Fails","% Rhat Fails","% bulk-ESS Fails"]
summary_stats = pd.concat([summary_stats,temp_df],axis=1)
summary_stats.to_csv(f"../results/dataframes/sampling/Thinning_AllStats.csv")

CPU times: user 13.9 s, sys: 43.6 s, total: 57.4 s
Wall time: 1min 13s


In [7]:
summary_stats

Geweke Fails  Rhat Fails  ESS-Bulk Fails  \
Thinning Factor Biomass Composition                                             
1               Consensus                     828         460             745   
                Derived                       775         627             762   
                Fitted                        771         268             726   
                Interpolated                  872         707             749   
                ScaledConsensus               848         301             661   
10              Consensus                     852         629             687   
                Derived                       642         206             539   
                Fitted                        882         589             632   
                Interpolated                  860         621             612   
                ScaledConsensus               871         234             562   
100             Consensus                     481          37               7   
                Derived                       508          29              16   
                Fitted                        520          24              13   
                Interpolated                  758         105              24   
                ScaledConsensus               463          18               0   
1000            Consensus                     344           4               0   
                Derived                       378           6               0   
                Fitted                        374           4               0   
                Interpolated                  201           2               0   
                ScaledConsensus               515           0               0   

                                     % Geweke Fails  % Rhat Fails  \
Thinning Factor Biomass Composition                                 
1               Consensus                 93.665158     52.036199   
                Derived                   86.206897     69.744160   
                Fitted                    86.922210     30.214205   
                Interpolated              95.929593     77.777778   
                ScaledConsensus           98.834499     35.081585   
10              Consensus                 93.729373     69.196920   
                Derived                   68.080594     21.845175   
                Fitted                    97.566372     65.154867   
                Interpolated              96.520763     69.696970   
                ScaledConsensus           95.925110     25.770925   
100             Consensus                 50.208768      3.862213   
                Derived                   56.381798      3.218646   
                Fitted                    55.614973      2.566845   
                Interpolated              80.467091     11.146497   
                ScaledConsensus           52.553916      2.043133   
1000            Consensus                 37.310195      0.433839   
                Derived                   41.086957      0.652174   
                Fitted                    39.872068      0.426439   
                Interpolated              22.358176      0.222469   
                ScaledConsensus           57.606264      0.000000   

                                     % bulk-ESS Fails  
Thinning Factor Biomass Composition                    
1               Consensus                   84.276018  
                Derived                     84.760845  
                Fitted                      81.848929  
                Interpolated                82.398240  
                ScaledConsensus             77.039627  
10              Consensus                   75.577558  
                Derived                     57.158006  
                Fitted                      69.911504  
                Interpolated                68.686869  
                ScaledConsensus             61.894273  
100             Consensus                    0.730689  
       

In [8]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
roi = ["G6PDH2","PGI","PYK","MDHm","CSm","AKGMALtm","NADH2_u6cm","NADH2_u6mh","CYOR_u6m","CYOOm","ATPS3m","ATPM"]
failures = failures[failures["Rxn"].isin(roi)]

failures.to_csv(f"../results/dataframes/sampling/Thinning_FailuresInterestingReactions.csv")
failures

Rxn Geweke Fails      Rhat  \
Thinning Factor Biomass Composition                                      
1               Consensus                   CSm            1  1.055778   
                Consensus            NADH2_u6cm            4  1.031601   
                Consensus                 CYOOm            4   1.02158   
                Consensus              CYOR_u6m            4  1.021581   
                Consensus              AKGMALtm            4  1.043052   
...                                         ...          ...       ...   
100             ScaledConsensus             PYK            4  1.014158   
                Interpolated                CSm            2  1.011679   
                Interpolated              CYOOm            4  1.020603   
                Interpolated           CYOR_u6m            4  1.020603   
                Interpolated                PYK            2  1.012712   

                                       ESS-Bulk     ESS-Tail  Rhat Fails  \
Thinning Factor Biomass Composition                                        
1               Consensus            140.745101    90.707419        True   
                Consensus             170.42466   215.500467        True   
                Consensus            210.925856   242.117459        True   
                Consensus            210.924266   242.117459        True   
                Consensus            227.631605   221.576272        True   
...                                         ...          ...         ...   
100             ScaledConsensus      810.792591  1752.854475        True   
                Interpolated         743.815296  1504.656838        True   
                Interpolated         790.206715   745.655611        True   
                Interpolated         790.207571   745.655611        True   
                Interpolated         751.092546  1932.148528        True   

                                     ESS-Bulk Fails  
Thinning Factor Biomass Composition                  
1               Consensus                      True  
                Consensus                      True  
                Consensus                      True  
                Consensus                      True  
                Consensus                      True  
...                                             ...  
100             ScaledConsensus               False  
                Interpolated                  False  
                Interpolated                  False  
                Interpolated                  False  
                Interpolated                  False  

[115 rows x 7 columns]

In [9]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
failures.loc[:,"Subsystem"] = failures["Rxn"].apply(lambda x:rgdict.get(x))

test = failures[(failures["Rhat Fails"] > 0)^(failures["ESS-Bulk Fails"] > 0)]
sum_fails_by_subsystem = test.groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()[["Rhat Fails", "ESS-Bulk Fails"]]
sum_fails_by_subsystem = sum_fails_by_subsystem.groupby(["Thinning Factor", "Subsystem"]).sum().unstack([0])
sum_fails_by_subsystem = sum_fails_by_subsystem.fillna(0)


sum_fails_by_subsystem.to_csv(f"../results/dataframes/sampling/Thinning_FailuresSubsystem.csv")
sum_fails_by_subsystem

Rhat Fails              \
Thinning Factor                                          1     10    100    
Subsystem                                                                   
Alanine And Aspartate Metabolism                          1.0   0.0   1.0   
Amino Sugar And Nucleotide Sugar Metabolism               0.0   0.0   1.0   
Arginine And Proline Metabolism                           0.0   3.0   0.0   
Biomass Composition                                       0.0   0.0   2.0   
Carbohydrate Metabolism                                   0.0   0.0   0.0   
Citric Acid Cycle                                         0.0   5.0   9.0   
Complex Alcohol Metabolism                                0.0   0.0   0.0   
Cysteine Metabolism                                       0.0   2.0   0.0   
Exchange Reaction                                         0.0   0.0   1.0   
Fatty Acid Biosynthesis                                   1.0   7.0  49.0   
Fatty Acid Degradation                                   35.0  23.0   7.0   
Folate Metabolism                                        14.0   0.0   9.0   
Fructose And Mannose Metabolism                           0.0   0.0   0.0   
Galactose Metabolism                                      0.0   0.0   0.0   
Glutamate Metabolism                                      1.0   2.0   1.0   
Glutamine Metabolism                                      0.0   0.0   0.0   
Glycerolipid Metabolism                                   2.0   5.0   0.0   
Glycine And Serine Metabolism                             2.0   2.0   3.0   
Glycolysis/Gluconeogenesis                                0.0   0.0   7.0   
Glycosylation Pathway                                     0.0   0.0   0.0   
Glyoxylate And Dicarboxylate Metabolism                   3.0   3.0   1.0   
Histidine Metabolism                                      3.0   0.0   0.0   
Inositol Phosphate Metabolism                             0.0   0.0   0.0   
Methanol Metabolism                                       0.0   0.0   1.0   
Methionine Metabolism                                     0.0  11.0   0.0   
N-Glycan Metabolism                                       0.0   0.0   2.0   
Nad Biosynthesis                                          1.0   4.0   3.0   
Nucleotide Metabolism                                    10.0  16.0  14.0   
Other                                                     0.0   0.0   0.0   
Other Amino Acid Metabolism                               1.0   2.0   4.0   
Oxidative Phosphorylation                                 1.0   0.0   3.0   
Pantothenate And Coa Biosynthesis                         0.0   4.0   8.0   
Pentose Phosphate Pathway                                 0.0   2.0   0.0   
Phospholipid Metabolism                                   1.0   3.0   1.0   
Porphyrin And Chlorophyll Metabolism                      0.0   0.0   0.0   
Pyridoxine Metabolism                                     7.0   7.0   4.0   
Pyruvate Metabolism                                       8.0  15.0   3.0   
Quinone Biosynthesis                                      0.0   0.0   0.0   
Recombinant Protein Production                            0.0   0.0   0.0   
Riboflavin Metabolism                                     0.0   0.0   6.0   
Sphingolipid Metabolism                                   4.0  35.0   0.0   
Sterol Metabolism                                         0.0   5.0   0.0   
Thiamine Metabolism                                       0.0   9.0   0.0   
Threonine And Lysine Metabolism                           2.0   2.0   0.0   
Transport, Er                                             0.0   0.0   2.0   
Transport, Extracellular                                  0.0   0.0   1.0   
Transport, Golgi Apparatus                                0.0   0.0   0.0   
Transport, Mitochondrial                                 13.0   6.0   7.0   
Transport, Nuclear                                        0.0   0.0   0.0   
Transport, Peroxisomal                         

In [10]:
rhat_fails = failures[failures["Rhat Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
rhat_fails = rhat_fails.groupby(["Thinning Factor", "Biomass Composition"])["Rhat Fails"].nlargest(5)
rhat_fails.index = rhat_fails.index.droplevel([0,1])
rhat_fails = rhat_fails.to_frame()

rhat_fails.to_csv(f"../results/dataframes/sampling/Thinning_FailuresRhat.csv")
rhat_fails

Rhat Fails
Thinning Factor Biomass Composition Subsystem                                             
1               Consensus           Fatty Acid Biosynthesis                             60
                                    Transport, Mitochondrial                            42
                                    Nucleotide Metabolism                               26
                                    Transport, Peroxisomal                              20
                                    Valine, Leucine, And Isoleucine Metabolism          19
...                                                                                    ...
1000            Fitted              Fatty Acid Biosynthesis                              1
                                    Sphingolipid Metabolism                              1
                                    Transport, Extracellular                             1
                Interpolated        Exchange Reaction                                    1
                                    Transport, Extracellular                             1

[89 rows x 1 columns]

In [11]:
essbulk_fails = failures[failures["ESS-Bulk Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
if not essbulk_fails.empty:
    sum_essbulk = essbulk_fails.groupby(["Thinning Factor", "Biomass Composition"])["ESS-Bulk Fails"].nlargest(5)
    sum_essbulk.index = sum_essbulk.index.droplevel([0,1])
    sum_essbulk = sum_essbulk.to_frame()
    sum_essbulk.to_csv(f"../results/dataframes/sampling/Thinning_FailuresSumESS.csv")
    
else:
    
    sum_essbulk = "No ESS Bulk fails"

sum_essbulk

ESS-Bulk Fails
Thinning Factor Biomass Composition Subsystem                                       
1               Consensus           Fatty Acid Biosynthesis                       77
                                    Transport, Mitochondrial                      58
                                    Sphingolipid Metabolism                       56
                                    Fatty Acid Degradation                        54
                                    Nucleotide Metabolism                         47
...                                                                              ...
100             Interpolated        Transport, Golgi Apparatus                     5
                                    Glycosylation Pathway                          3
                                    Transport, Peroxisomal                         3
                                    Alanine And Aspartate Metabolism               2
                                    Exchange Reaction                              2

[70 rows x 1 columns]

In [12]:
geweke_fails = failures[failures["Geweke Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).count()
geweke_fails = geweke_fails.groupby(["Thinning Factor", "Biomass Composition"])["Geweke Fails"].nlargest(5)
geweke_fails.index = geweke_fails.index.droplevel([0,1])
geweke_fails = geweke_fails.to_frame()


geweke_fails.to_csv(f"../results/dataframes/sampling/Thinning_FailuresGeweke.csv")
geweke_fails

Geweke Fails
Thinning Factor Biomass Composition Subsystem                             
1               Consensus           Fatty Acid Biosynthesis             75
                                    Transport, Mitochondrial            58
                                    Sphingolipid Metabolism             57
                                    Nucleotide Metabolism               46
                                    Sterol Metabolism                   33
...                                                                    ...
1000            Fitted              Fatty Acid Biosynthesis              1
                                    Sphingolipid Metabolism              1
                                    Transport, Extracellular             1
                Interpolated        Exchange Reaction                    1
                                    Transport, Extracellular             1

[89 rows x 1 columns]

### Additional diagnostic plots of a selection of reactions (those used in later analysis) - each inspected as part of convergence assessment
Additional plots include Geweke plots, Rank plots, local, quantile and evolution ESS plots

In [13]:
%%time

from matplotlib.backends.backend_pdf import PdfPages

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

n_chains = 4
interesting_reactions = ["G6PDH2","PGI","PC","PYK","MDHm", 
                         "CSm", "AKGMALtm","NADH2_u6cm","NADH2_u6mh",
                         "SUCD2_u6m","CYOR_u6m","CYOOm","ATPS3m"]

for rxn in interesting_reactions:

    with PdfPages(f'../results/plots/SamplingDiagnostics{rxn}.pdf') as pdf:
        
        for equation in ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]:
            
            fig = plt.figure(figsize=(20, 18), constrained_layout=True)
            spec = fig.add_gridspec(nrows=4, ncols=5, height_ratios=[1,1,1,1], hspace=0.1, bottom=0.1)
            
            for i, tfactor in enumerate([1,10,100, 1000]):
                
                chains_df = pd.read_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz")
                chains_df = [chains_df.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
                
                grouped, rxns = extractchains(chains_df)
                
                chains = grouped[rxns.index(rxn)]
                cmap = plt.cm.get_cmap('tab10', 10)
                color_list = [matplotlib.colors.rgb2hex(cmap(j)[:3]) for j in range(cmap.N)][:4]

                axes = [fig.add_subplot(spec[i, j]) for j in np.arange(5)]

                plottrace(chains,axes[0],color_list)
                axes[0].set_ylabel("Trace Plot",fontsize=15)
                
                plotrank(chains,axes[1],color_list)
                axes[1].set_ylabel("Rank Plot",fontsize=15)
                
                az.plot_ess(chains,ax=axes[2], relative=True, kind="local")
                az.plot_ess(chains,ax=axes[3], relative=True, kind="quantile")
                az.plot_ess(chains,ax=axes[4], relative=True, kind="evolution")
                
                [axes[i].set_ylim(0,1.4) for i in [2,3,4]]
                [axes[i].set_title("") for i in [1,2,3,4]]
                [ax.tick_params(labelsize=15) for ax in axes]

                axes[2].set_title(f"{rxn} - {biocomp} - Thinning factor {tfactor}",fontsize=20, pad=30)

            pdf.savefig(fig)
            plt.close()
                    
               

CPU times: user 3min 17s, sys: 8.71 s, total: 3min 26s
Wall time: 3min 26s
